# Imports

In [325]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split

# Data preprocessing

In [326]:
results = pd.read_csv('csv_data/results.csv')

In [327]:
results.head(10)

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,E0,14/08/10,Aston Villa,West Ham,3.0,0.0,H,2.0,0.0,H,...,11.0,2.0,15.0,15.0,16.0,7.0,1.0,2.0,0.0,0.0
1,E0,14/08/10,Blackburn,Everton,1.0,0.0,H,1.0,0.0,H,...,2.0,12.0,19.0,14.0,1.0,3.0,2.0,1.0,0.0,0.0
2,E0,14/08/10,Bolton,Fulham,0.0,0.0,D,0.0,0.0,D,...,9.0,7.0,12.0,13.0,4.0,8.0,1.0,3.0,0.0,0.0
3,E0,14/08/10,Chelsea,West Brom,6.0,0.0,H,2.0,0.0,H,...,13.0,4.0,10.0,10.0,3.0,1.0,1.0,0.0,0.0,0.0
4,E0,14/08/10,Sunderland,Birmingham,2.0,2.0,D,1.0,0.0,H,...,2.0,7.0,13.0,10.0,3.0,6.0,3.0,3.0,1.0,0.0
5,E0,14/08/10,Tottenham,Man City,0.0,0.0,D,0.0,0.0,D,...,18.0,7.0,13.0,16.0,10.0,3.0,0.0,2.0,0.0,0.0
6,E0,14/08/10,Wigan,Blackpool,0.0,4.0,A,0.0,3.0,A,...,6.0,7.0,8.0,11.0,6.0,4.0,1.0,1.0,0.0,0.0
7,E0,14/08/10,Wolves,Stoke,2.0,1.0,H,2.0,0.0,H,...,7.0,6.0,17.0,13.0,5.0,5.0,0.0,2.0,0.0,0.0
8,E0,15/08/10,Liverpool,Arsenal,1.0,1.0,D,0.0,0.0,D,...,4.0,7.0,13.0,15.0,9.0,11.0,1.0,3.0,1.0,1.0
9,E0,16/08/10,Man United,Newcastle,3.0,0.0,H,2.0,0.0,H,...,10.0,3.0,9.0,5.0,5.0,3.0,2.0,2.0,0.0,0.0


In [328]:
results.loc[results['Div'] == 'E0', 'Div'] = 'E1'    # first English division should be called 'E1' instead of 'E0'

## Handling missing values

In [329]:
results[results['HY'].isnull()]

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6969,I1,23/09/12,Cagliari,Roma,0.0,3.0,A,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7945,I1,02/03/15,Roma,Juventus,1.0,1.0,D,0.0,0.0,D,...,3.0,1.0,11.0,16.0,1.0,5.0,NaN,5.0,1.0,0.0
8078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [330]:
results = results.dropna(axis='index', how='all')  # drop all rows that only contain NaN values

In [331]:
results.loc[7945, 'HY'] = 6    # add the missing value by hand

In [332]:
results.drop(labels=6969, axis='index', inplace=True)  # remove row that misses too many values

In [333]:
results[results['HTHG'].isnull()]

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
8477,I1,28/08/16,Sassuolo,Pescara,0.0,3.0,A,NaN,NaN,NaN,...,3.0,5.0,12.0,21.0,2.0,7.0,2.0,2.0,0.0,0.0


In [334]:
results.drop(labels=8477, axis='index', inplace=True)  # remove game with non-representative result

In [335]:
# all missing values have been dealt with by this point
for col in results.columns:
    print(col, len(results[results[col].isnull()]))

Div 0
Date 0
HomeTeam 0
AwayTeam 0
FTHG 0
FTAG 0
FTR 0
HTHG 0
HTAG 0
HTR 0
HS 0
AS 0
HST 0
AST 0
HF 0
AF 0
HC 0
AC 0
HY 0
AY 0
HR 0
AR 0


## Datatypes

In [336]:
# convert float columns to int
for col in results.columns:
    if results[col].dtype == np.float64:
        results[col] = results[col].astype(int)

In [337]:
results.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,E1,14/08/10,Aston Villa,West Ham,3,0,H,2,0,H,...,11,2,15,15,16,7,1,2,0,0
1,E1,14/08/10,Blackburn,Everton,1,0,H,1,0,H,...,2,12,19,14,1,3,2,1,0,0
2,E1,14/08/10,Bolton,Fulham,0,0,D,0,0,D,...,9,7,12,13,4,8,1,3,0,0
3,E1,14/08/10,Chelsea,West Brom,6,0,H,2,0,H,...,13,4,10,10,3,1,1,0,0,0
4,E1,14/08/10,Sunderland,Birmingham,2,2,D,1,0,H,...,2,7,13,10,3,6,3,3,1,0


In [338]:
# convert Date column to proper datetime format
results['Date'] = pd.to_datetime(results['Date']) #.dt.strftime('%d/%m/%Y')    

In [339]:
results.sample(5)

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
6255,I1,2010-10-29,Genoa,Inter,0,1,A,0,1,A,...,6,5,10,9,5,5,2,2,0,0
10910,SP1,2014-04-01,Almeria,Granada,3,0,H,2,0,H,...,8,3,18,10,9,9,3,3,0,0
7525,I1,2014-01-26,Sampdoria,Bologna,1,1,D,0,0,D,...,5,5,16,11,6,5,4,1,0,0
9702,SP1,2010-11-13,Barcelona,Villarreal,3,1,H,1,1,D,...,5,2,22,19,4,4,2,3,0,0
8062,I1,2015-05-24,Empoli,Sampdoria,1,1,D,0,0,D,...,4,5,12,13,4,5,3,3,0,0


In [340]:
results.dtypes

Div                 object
Date        datetime64[ns]
HomeTeam            object
AwayTeam            object
FTHG                 int32
FTAG                 int32
FTR                 object
HTHG                 int32
HTAG                 int32
HTR                 object
HS                   int32
AS                   int32
HST                  int32
AST                  int32
HF                   int32
AF                   int32
HC                   int32
AC                   int32
HY                   int32
AY                   int32
HR                   int32
AR                   int32
dtype: object

In [341]:
# Assign an ID to each team for easier identification
team_ids = defaultdict()

assert len(results['HomeTeam'].unique()) == len(results['AwayTeam'].unique())

teams = results['HomeTeam'].unique()

index = 0
for team in teams:
    team_ids[team] = index
    print(index, team)    
    index += 1



0 Aston Villa
1 Blackburn
2 Bolton
3 Chelsea
4 Sunderland
5 Tottenham
6 Wigan
7 Wolves
8 Liverpool
9 Man United
10 Arsenal
11 Birmingham
12 Everton
13 Stoke
14 West Brom
15 West Ham
16 Fulham
17 Newcastle
18 Man City
19 Blackpool
20 QPR
21 Swansea
22 Norwich
23 Reading
24 Southampton
25 Crystal Palace
26 Hull
27 Cardiff
28 Leicester
29 Burnley
30 Bournemouth
31 Watford
32 Middlesbrough
33 Brighton
34 Huddersfield
35 Bayern Munich
36 FC Koln
37 Freiburg
38 Hamburg
39 Hannover
40 Hoffenheim
41 M'gladbach
42 Dortmund
43 Mainz
44 Kaiserslautern
45 Ein Frankfurt
46 Nurnberg
47 Schalke 04
48 St Pauli
49 Werder Bremen
50 Wolfsburg
51 Leverkusen
52 Stuttgart
53 Augsburg
54 Hertha
55 Greuther Furth
56 Fortuna Dusseldorf
57 Braunschweig
58 Paderborn
59 Darmstadt
60 Ingolstadt
61 RB Leipzig
62 Roma
63 Udinese
64 Bari
65 Chievo
66 Fiorentina
67 Milan
68 Palermo
69 Parma
70 Sampdoria
71 Bologna
72 Cagliari
73 Cesena
74 Inter
75 Brescia
76 Catania
77 Genoa
78 Juventus
79 Lazio
80 Lecce
81 Napoli
82 

In [342]:
# Convert team name columns to IDs
#results['HomeID'] = results['HomeTeam']
#results['AwayID'] = results['AwayTeam']
results.rename(columns={'HomeTeam': 'HomeID', 'AwayTeam': 'AwayID'}, inplace=True)
results['HomeID'].replace(team_ids, inplace=True)
results['AwayID'].replace(team_ids, inplace=True)

In [343]:
results.sample(6)

,Div,Date,HomeID,AwayID,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
9514,I1,2019-03-04,62,66,2,2,D,1,1,D,...,6,5,15,15,4,6,3,4,0,0
8670,I1,2017-01-29,93,90,4,1,H,1,1,D,...,7,3,13,14,8,1,3,1,0,0
5768,D1,2018-02-18,41,42,0,1,A,0,1,A,...,11,2,15,16,6,5,3,3,0,0
924,E1,2012-12-15,20,16,2,1,H,0,0,D,...,9,3,10,14,3,3,1,2,0,0
9483,I1,2019-10-03,66,79,1,1,D,0,1,A,...,1,6,8,14,4,7,3,1,0,0
12495,SP1,2018-02-18,117,108,3,5,A,2,1,H,...,5,9,7,15,7,5,3,5,0,0


Preprocessing should be finished now.

# Data extraction into new Dataframe

In [344]:
# We don't need HomeID and AwayID here, but I put it in just for exploration
df = results[['HomeID', 'AwayID', 'FTHG', 'FTAG', 'FTR']].copy()
df

,HomeID,AwayID,FTHG,FTAG,FTR
0,0,15,3,0,H
1,1,12,1,0,H
2,2,16,0,0,D
3,3,14,6,0,H
4,4,11,2,2,D
...,...,...,...,...,...
13015,97,105,2,2,D
13016,111,106,2,0,H
13017,120,109,0,2,A
13018,122,107,2,2,D


In [345]:
# Replace H, D and A by 1, 0 and 2 (target variables)
result_dict = {'H': 1, 'D': 0, 'A': 2}
df['FTR'].replace(result_dict, inplace=True)

In [346]:
#slc = df['AwayID'] == 105
#df[slc][['FTR']].value_counts() / sum(slc)

In [351]:
# The column H5GR and A5GR contain the points collected by the home
# and away teams over the last 5 games.
# (Win = 3 points, Draw = 1 point, Loss = 0 points)
# If there is not enough data, we just add 1.333333 points (or some mean maybe)
# until we get to 5 games.
df['H5GR'] = 0
df['A5GR'] = 0

cur_game = 0
for game in results.iterrows():
    team_id = game[1]['HomeID']
    team_points = 0         # points from the last 5 games
    
    recent_games = results.loc[(results['HomeID'] == team_id) | (results['AwayID'] == team_id)].tail(6) # TODO Date checken!
    recent_games.drop(recent_games.tail(1).index, inplace=True)   # drop cur_game from recent games
    
    # Add the points accordingly
    for recent in recent_games:
        if recent['FTR'] == 'D':
            team_points += 1
        elif (recent['FTR'] == '1' and recent['HomeID'] == team_id
            or recent['FTR'] == '2' and recent['AwayID'] == team_id):
            team_points += 3           
        # TODO funktioniert noch nicht

    
    df['H5GR'][cur_game] = team_points
    
    cur_game += 1


TypeError: string indices must be integers

In [348]:
df.sample(3)

,HomeID,AwayID,FTHG,FTAG,FTR,H5GR,A5GR
7131,69,78,1,1,0,0,0
6857,73,71,0,0,0,0,0
5948,61,47,0,0,0,0,0
